In [6]:
import yaml
import pprint
from census import Census
from us import states
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!pip install census

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 970.1 kB/s eta 0:00:000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.2/123.2 kB 2.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 3.0 MB/s eta 0:00:0000:01


In [4]:
!pip install us

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for us: filename=us-2.0.2-py3-none-any.whl size=11940 sha256=54516212e72f5db05c4d8435215230aa74ea6c85aa75129e2b041de43ba7485f
  Stored in directory: /n/home_fasse/nsardana/.cache/pip/wheels/b0/2a/8b/46c38c4bc2d65459d10a393292c197faf7d1071bc6177f84a4
  Created wheel for jellyfish: filename=jellyfish-0.6.1-cp310-cp310-linux_x86_64.whl size=23751 sha256=56bc8872b755f4aab48922253b631274d750e5bc0121b3f7d73f4db87e959230
  Stored in directory: /n/home_fasse/nsardana/.cache/pip/wheels/b4/50/f2/2a1bd2c756df70adc1f703e09453b6f68255385789b32107ae
Successfully built us jellyfish


In [9]:
#setting up api key
key = '0fec0de3c4e525041f1101feb0c3a41be9b4a007'

In [127]:
#reading the yaml file
file_name = "../data/input/remote/census.yaml"
with open(file_name) as file:
    census_doc = yaml.safe_load(file)

In [48]:
def get_census_zcta(var_name, var_entry, year):
    c = Census(key)
    if 'den' in var_entry:
        den_list = var_entry['den']
        num_list = var_entry['num']
        all_var_list = den_list + num_list
        data = c.acs5.get(all_var, {'for': 'zip code tabulation area:*', 'year': year})
        df = pd.DataFrame.from_records(data)  
        df[var_name] = df[num_list].sum(axis=1) / df[den_list].sum(axis=1)
    else:
        num_list = var_entry['num']
        all_var_list = num_list
        data = c.acs5.get(all_var, {'for': 'zip code tabulation area:*', 'year': year})
        df = pd.DataFrame.from_records(data)  
        df[var_name] = df[num_list].sum(axis=1) / df[den_list].sum(axis=1)
        
    df.rename(columns={'zip code tabulation area':'zcta'}, inplace = True)
    col_list = ['zcta', var_name]
    df = df[col_list]
    return df
        

def main(args):
    # read in census yaml ----
    census = pd.read_yaml(args.census_yaml)
    
    df_list = []
    
    # get census data for all zcta for each variables----
    for variable in census.keys():
        var_entry = document[variable][args.year]
        df_var = get_census_zcta(variable, var_entry, args.year)
        df_list.append(df_var)

    # merge dataframes ----
    df = pd.concat(df_list)

    # write ouput ----
    df.set_index('zcta', inplace=True)

    output_file = f"{args.output_prefix}_{args.year}.parquet"
    if args.output_format == 'csv':
        df.to_csv(output_file)
    elif args.output_format == 'parquet':
        df.to_parquet(output_file)

In [22]:
df_list = []
for variable in document.keys():
    document[variable]
    

pct_blk
medhouseholdincome
pct_owner_occ
hispanic
education


In [31]:
entry = document['pct_blk'][2011]
entry['num'] + entry['den']

['B02001_003E', 'B02001_001E']

In [36]:
for variable in document.keys():
        entry = document[variable][2011]
        print(entry)
        #print(entry)


{'prefix': 'ACS11_5yr_', 'num': ['B02001_003E'], 'den': ['B02001_001E']}
{'prefix': 'ACS11_5yr_', 'num': ['B19013_001E']}
{'prefix': 'ACS11_5yr_', 'num': ['B11012_004E', 'B11012_008E', 'B11012_011E', 'B11012_014E'], 'den': ['B11012_001E']}
{'prefix': 'ACS11_5yr_', 'num': ['B03003_003E'], 'den': ['B03003_001E']}
{'prefix': 'ACS11_5yr_', 'num': ['B15001_036E', 'B15001_037E', 'B15001_077E', 'B15001_078E'], 'den': ['B15001_035E', 'B15001_076E']}


In [113]:
def get_census_zcta(var_name, var_entry, year):
    key = '0fec0de3c4e525041f1101feb0c3a41be9b4a007'
    c = Census(key)
    print(var_name)
    if 'den' in var_entry:
        den_list = var_entry['den']
        num_list = var_entry['num']
        all_var_list = den_list + num_list
        data = c.acs5.get(all_var_list, {'for': 'zip code tabulation area:*', 'year': year})
        df = pd.DataFrame.from_records(data) 
        df[var_name] = df[num_list].sum(axis=1) / df[den_list].sum(axis=1)
    else:
        num_list = var_entry['num']
        all_var_list = num_list
        data = c.acs5.get(all_var_list, {'for': 'zip code tabulation area:*', 'year': year})
        df = pd.DataFrame.from_records(data)  
        df[var_name] = df[num_list].sum(axis=1)
        
    df.rename(columns={'zip code tabulation area':'zcta'}, inplace = True)
    col_list = ['zcta', var_name]
    df = df[col_list]
    df.set_index('zcta', inplace=True)
    return df
        

In [124]:
#census = pd.read_yaml(args.census_yaml)

df_list = []

# get census data for all zcta for each variables----
for variable in census.keys():
    var_entry = census_doc[variable][2011]
    df_var = get_census_zcta(variable, var_entry, 2011)
    df_list.append(df_var)

# merge dataframes ----
df = pd.concat(df_list, axis=1)
df.reset_index(inplace=True)

pct_blk
medhouseholdincome
pct_owner_occ
hispanic
education


In [125]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33774 entries, 0 to 33773
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   zcta                33774 non-null  object 
 1   pct_blk             33184 non-null  float64
 2   medhouseholdincome  33774 non-null  float64
 3   pct_owner_occ       32812 non-null  float64
 4   hispanic            33184 non-null  float64
 5   education           32321 non-null  float64
dtypes: float64(5), object(1)
memory usage: 1.5+ MB


In [123]:
df.columns

Index(['pct_blk', 'medhouseholdincome', 'pct_owner_occ', 'hispanic',
       'education'],
      dtype='object')

In [105]:
len(data)

33120

In [68]:
df[num_list].sum(axis=1)

KeyError: "None of [Index(['B02001_003E'], dtype='object')] are in the [columns]"

In [79]:
c = Census(key, year=2011)
variable = 'pct_blk'
var_entry = census[variable][2011]

den_list = var_entry['den']
num_list = var_entry['num']
print(num_list)
print(den_list)

all_var_list = den_list + num_list
data = c.acs5.get(all_var_list, {'for': 'zip code tabulation area:*', 'year': 2011})
df = pd.DataFrame.from_records(data) 




['B02001_003E']
['B02001_001E']


In [80]:
all_var_list


['B02001_001E', 'B02001_003E']

In [81]:
df

,B02001_001E,B02001_003E,state,zip code tabulation area
0,18533.0,261.0,72,00601
1,41930.0,1794.0,72,00602
2,54475.0,1767.0,72,00603
3,6386.0,43.0,72,00606
4,29111.0,700.0,72,00610
...,...,...,...,...
33115,116.0,0.0,02,99923
33116,689.0,0.0,02,99925
33117,1488.0,7.0,02,99926
33118,67.0,0.0,02,99927
